In [2]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import pandas as pd
import numpy as np
from itertools import chain

# Hi-C utilities imports:
import cooler
import bioframe
import cooltools
from cooltools.lib.numutils import fill_diag
from cooltools import insulation
from packaging import version
if version.parse(cooltools.__version__) < version.parse('0.5.2'):
    raise AssertionError("tutorials rely on cooltools version 0.5.2 or higher,"+
                         "please check your cooltools version and update to the latest")

# Visualization imports:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter

# helper functions for plotting
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    """format ticks with genomic coordinates as human readable"""
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)
        
# Functions to help with plotting
def pcolormesh_45deg(ax, matrix_c, start=0, resolution=1, *args, **kwargs):
    start_pos_vector = [start+resolution*i for i in range(len(matrix_c)+1)]
    import itertools
    n = matrix_c.shape[0]
    t = np.array([[1, 0.5], [-1, 0.5]])
    matrix_a = np.dot(np.array([(i[1], i[0])
                                for i in itertools.product(start_pos_vector[::-1],
                                                           start_pos_vector)]), t)
    x = matrix_a[:, 1].reshape(n + 1, n + 1)
    y = matrix_a[:, 0].reshape(n + 1, n + 1)
    im = ax.pcolormesh(x, y, np.flipud(matrix_c), *args, **kwargs)
    im.set_rasterized(True)
    return im

from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)
        
# create a functions that would return a series of rectangles around called dots
# in a specific region, and exposing importnat plotting parameters
def rectangles_around_dots(dots_df, region, loc="upper", lw=1, ec="cyan", fc="none"):
    """
    yield a series of rectangles around called dots in a given region
    """
    # select dots from the region:
    df_reg = bioframe.select(
        bioframe.select(dots_df, region, cols=("chrom1","start1","end1")),
        region,
        cols=("chrom2","start2","end2"),
    )
    rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # draw rectangular "boxes" around pixels called as dots in the "region":
    for s1, s2, e1, e2 in df_reg[["start1", "start2", "end1", "end2"]].itertuples(index=False):
        width1 = e1 - s1
        width2 = e2 - s2
        if loc == "upper":
            yield patches.Rectangle((s2, s1), width2, width1, **rectangle_kwargs)
        elif loc == "lower":
            yield patches.Rectangle((s1, s2), width1, width2, **rectangle_kwargs)
        else:
            raise ValueError("loc has to be uppper or lower")

In [7]:
import cooltools.lib.plotting

file_ref = '0hr'
files = ['0hr', '24hrs', '72hrs']

#files = ['0hr', '24hrs', '72hrs', 'WTCI', 'KOCI', 'WTCIR', 'KOCIR']

for file in files:
    flank=300_000
    
    print(file)
    res_kb = 10
    resname = f'{res_kb}kb'
    resval = f'{res_kb}000'
    
    data_dir = '../cool/'
    cool_file = data_dir + file + '.' + resname + '.mcool'
    clr = cooler.Cooler(cool_file + '::/resolutions/' + resval)
    resolution = clr.binsize
    
    cool_file_ref = data_dir + file_ref + '.' + resname + '.mcool'
    clr_ref = cooler.Cooler(cool_file_ref + '::/resolutions/' + resval)
    
    # Use bioframe to fetch the genomic features from the UCSC.
    hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
    hg38_cens = bioframe.fetch_centromeres('hg38')
    hg38_arms = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
    
    with open('../cool/' + file + '.' + resname + '.pkl', 'rb') as f:
        expected, insulation_table = pickle.load(f)
    
    with open('../cool/' + file_ref + '.' + resname + '.pkl', 'rb') as f:
        expected_ref, insulation_table_ref = pickle.load(f)
    
    # TADs from 300-400 kb
    DEG = bioframe.read_table('../bed/TAD.300-400kb.bed', schema='bed').query(f'chrom in {clr.chromnames}')
    DEG['mid'] = (DEG.end+DEG.start)//2
    
    stack = cooltools.pileup(clr, DEG, view_df=hg38_arms, flank=300_000)
    mask = np.array(DEG.strand == '-', dtype=bool)
    stack[:, :, mask] = stack[::-1, ::-1, mask]
    mtx_qry = np.nanmean(stack, axis=2)
    
    stack_ref = cooltools.pileup(clr_ref, DEG, view_df=hg38_arms, flank=300_000)
    mask_ref = np.array(DEG.strand == '-', dtype=bool)
    stack_ref[:, :, mask_ref] = stack_ref[::-1, ::-1, mask_ref]
    mtx_ref = np.nanmean(stack_ref, axis=2)
    
    mtx = np.divide(mtx_qry, mtx_ref)
    
    plt.imshow(
        np.log2(mtx),
        vmin = -0.3,
        vmax = 0.3,
        cmap='coolwarm',
        interpolation='none')
    plt.colorbar(label = 'log2 mean obs/exp difference')
    ticks_pixels = np.linspace(0, flank*2//resolution,5)
    ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*resolution//1000).astype(int)
    plt.xticks(ticks_pixels, ticks_kbp)
    plt.yticks(ticks_pixels, ticks_kbp)
    plt.xlabel('relative position, kbp')
    plt.ylabel('relative position, kbp')

    plt.savefig('../plot/' + file + '.' + file_ref + '.TAD.300-400kb.expected.subtraction.' + resname + '.pdf')
    plt.clf()
        
    print(file + ': completed\n')

0hr
0hr: completed

24hrs
24hrs: completed

72hrs
72hrs: completed



<Figure size 640x480 with 0 Axes>

In [8]:
import cooltools.lib.plotting

file_ref = 'WTCI'
files = ['WTCI', 'KOCI', 'WTCIR', 'KOCIR']

#files = ['0hr', '24hrs', '72hrs', 'WTCI', 'KOCI', 'WTCIR', 'KOCIR']

for file in files:
    flank=300_000
    
    print(file)
    res_kb = 10
    resname = f'{res_kb}kb'
    resval = f'{res_kb}000'
    
    data_dir = '../cool/'
    cool_file = data_dir + file + '.' + resname + '.mcool'
    clr = cooler.Cooler(cool_file + '::/resolutions/' + resval)
    resolution = clr.binsize
    
    cool_file_ref = data_dir + file_ref + '.' + resname + '.mcool'
    clr_ref = cooler.Cooler(cool_file_ref + '::/resolutions/' + resval)
    
    # Use bioframe to fetch the genomic features from the UCSC.
    hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
    hg38_cens = bioframe.fetch_centromeres('hg38')
    hg38_arms = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
    
    with open('../cool/' + file + '.' + resname + '.pkl', 'rb') as f:
        expected, insulation_table = pickle.load(f)
    
    with open('../cool/' + file_ref + '.' + resname + '.pkl', 'rb') as f:
        expected_ref, insulation_table_ref = pickle.load(f)
    
    # TADs from 300-400 kb
    DEG = bioframe.read_table('../bed/TAD.300-400kb.bed', schema='bed').query(f'chrom in {clr.chromnames}')
    DEG['mid'] = (DEG.end+DEG.start)//2
    
    stack = cooltools.pileup(clr, DEG, view_df=hg38_arms, flank=300_000)
    mask = np.array(DEG.strand == '-', dtype=bool)
    stack[:, :, mask] = stack[::-1, ::-1, mask]
    mtx_qry = np.nanmean(stack, axis=2)
    
    stack_ref = cooltools.pileup(clr_ref, DEG, view_df=hg38_arms, flank=300_000)
    mask_ref = np.array(DEG.strand == '-', dtype=bool)
    stack_ref[:, :, mask_ref] = stack_ref[::-1, ::-1, mask_ref]
    mtx_ref = np.nanmean(stack_ref, axis=2)
    
    mtx = np.divide(mtx_qry, mtx_ref)
    
    plt.imshow(
        np.log2(mtx),
        vmin = -0.3,
        vmax = 0.3,
        cmap='coolwarm',
        interpolation='none')
    plt.colorbar(label = 'log2 mean obs/exp difference')
    ticks_pixels = np.linspace(0, flank*2//resolution,5)
    ticks_kbp = ((ticks_pixels-ticks_pixels[-1]/2)*resolution//1000).astype(int)
    plt.xticks(ticks_pixels, ticks_kbp)
    plt.yticks(ticks_pixels, ticks_kbp)
    plt.xlabel('relative position, kbp')
    plt.ylabel('relative position, kbp')

    plt.savefig('../plot/' + file + '.' + file_ref + '.TAD.300-400kb.expected.subtraction.' + resname + '.pdf')
    plt.clf()
        
    print(file + ': completed\n')

WTCI
WTCI: completed

KOCI
KOCI: completed

WTCIR
WTCIR: completed

KOCIR
KOCIR: completed



<Figure size 640x480 with 0 Axes>